In [ ]:
import numpy as np
import albumentations as A
import random
import os
import cv2
import matplotlib.pyplot as plt

# Set the random seed using matriculation number
matriculation_number =   # Replace with your actual matriculation number
random.seed(matriculation_number)
np.random.seed(matriculation_number)

# Set the path to the extracted MiniBAGLS dataset
extract_path = '' # Replace it with actual path

# List all images in the dataset
dataset_files = os.listdir(extract_path)
images = [f for f in dataset_files if f.endswith('.jpg') or f.endswith('.png') or f.isdigit()]

# Check if images are available
if len(images) == 0:
    raise ValueError("No images found in the dataset. Please check the file path and naming conventions.")

# Choose a random image and its corresponding mask and metadata
random_index = np.random.choice(len(images))
image_name = images[random_index]
image_path = os.path.join(extract_path, image_name)
mask_path = os.path.join(extract_path, f"{os.path.splitext(image_name)[0]}_seg.png")
metadata_path = os.path.join(extract_path, f"{image_name}.meta")

# Load the image and mask
image = cv2.imread(image_path)
if image is None:
    raise FileNotFoundError(f"Image not found at path: {image_path}")

mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
if mask is None:
    raise FileNotFoundError(f"Mask not found at path: {mask_path}")

# Resize the mask to match the image dimensions if necessary
if image.shape[:2] != mask.shape[:2]:
    mask = cv2.resize(mask, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)

# Convert mask to numpy array
mask = np.array(mask)

# Convert BGR to RGB for visualization
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Define a set of augmentations using albumentations
augmentations = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.GaussianBlur(blur_limit=(3, 7), p=0.3),
    A.ElasticTransform(p=0.3),
    A.RandomBrightnessContrast(p=0.4),
])

# Apply the augmentations
augmented = augmentations(image=image, mask=mask)
augmented_image = augmented['image']
augmented_mask = augmented['mask']

# Plot the original and augmented image and mask
fig, ax = plt.subplots(2, 2, figsize=(10, 10))

# Original image
ax[0, 0].imshow(image)
ax[0, 0].set_title('Original Image')
ax[0, 0].axis('off')

# Original mask
ax[0, 1].imshow(mask, cmap='gray')
ax[0, 1].set_title('Original Mask')
ax[0, 1].axis('off')

# Augmented image
ax[1, 0].imshow(augmented_image)
ax[1, 0].set_title('Augmented Image')
ax[1, 0].axis('off')

# Augmented mask
ax[1, 1].imshow(augmented_mask, cmap='gray')
ax[1, 1].set_title('Augmented Mask')
ax[1, 1].axis('off')

plt.tight_layout()
plt.show()